# Real-time Implementation and Automation with IBKR

--------------------------------------------------------------------------------------------------------------------

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with an IBKR Practice/Demo Account and NOT with a Live Trading Account._

------------------------------------------------------------------------------------

## IBKR API - Recap

__Update__: ib_insync has been migrated to __ib_async__. From now on, __please use ib_async:__

In [1]:
!pip install ib_async --break-system-packages

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from ib_async import *
util.startLoop() 

In [3]:
ib = IB()

In [4]:
ib.connect()

API connection failed: ConnectionRefusedError(111, "Connect call failed ('127.0.0.1', 7497)")
Make sure API port on TWS/IBG is open


ConnectionRefusedError: [Errno 111] Connect call failed ('127.0.0.1', 7497)

In [ ]:
ib.positions()

In [ ]:
contract = Forex('EURUSD')
contract

In [ ]:
ib.qualifyContracts(contract)

In [ ]:
data = ib.reqMktData(contract) # subscribe to tick data
data

In [ ]:
data

In [ ]:
data.bid

In [ ]:
ib.cancelMktData(contract) # unsubscribe market data

In [ ]:
ib.reqMktData(contract)

In [ ]:
ticker = ib.ticker(contract)
ticker

In [ ]:
ticker.bid

In [ ]:
ib.cancelMktData(contract) # unsubscribe market data

In [ ]:
ib.disconnect()

## Streaming Tick Data

In [ ]:
from ib_async import *
import pandas as pd
util.startLoop() 
ib = IB()

In [ ]:
ib.connect()

In [ ]:
contract = Forex('EURUSD')
ib.reqMktData(contract)
ticker = ib.ticker(contract)

In [ ]:
ticker

In [ ]:
def onPendingTickers(tickers): # what shall happen after receiving a new tick
    global message
    message = "time: {} | Bid: {} | Ask:{}".format(ticker.time, ticker.bid, ticker.ask)
    print(message, end = '\r')

In [ ]:
ib.pendingTickersEvent += onPendingTickers # activate onPendingTickers
ib.sleep(30) # new, to be added!!!

In [ ]:
message # still getting updated

In [ ]:
ib.pendingTickersEvent -= onPendingTickers # de-activate onPendingTickers

In [ ]:
ticker

In [ ]:
ib.cancelMktData(contract)

In [ ]:
ib.disconnect()

## Streaming Tick Data for multiple Symbols

In [ ]:
from ib_async import *
import pandas as pd
from IPython.display import display, clear_output
util.startLoop() 
ib = IB()

In [ ]:
ib.connect()

In [ ]:
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD')]
ib.qualifyContracts(*contracts)

In [ ]:
for contract in contracts:
    ib.reqMktData(contract)

In [ ]:
df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])
df

In [ ]:
def onPendingTickers(tickers): # what shall happen after receiving a new tick
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df) 

In [ ]:
ib.pendingTickersEvent += onPendingTickers # activate onPendingTickers
ib.sleep(30) # new to be added!!!

In [ ]:
df # still getting updated

In [ ]:
ib.pendingTickersEvent -= onPendingTickers # de-activate onPendingTickers

In [ ]:
for contract in contracts:
    ib.cancelMktData(contract)

In [ ]:
ib.disconnect()

## Streaming Real Time Bars (and Historical)

- Trading Stratgies use bar data (uniform time periods with ohlc data), not tick data (non-constant frequency, based on trades and quotes)
- An SMA Crossover 50/200 (days) Strategy works with daily bars.
- Day Trading Strategies are based on e.g. [1min, 5min, 20min, 1h, 3h, 6h] bars
- Bar Size / Frequency itself is a Strategy Parameter (to be optimized)

In [ ]:
from ib_async import *
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
util.startLoop() 

In [ ]:
ib = IB()
ib.connect()

In [ ]:
contract = Forex('EURUSD')
contract

__durationStr__: Time span of all the bars. Examples:
        __'60 S', '30 D', '13 W', '6 M', '10 Y'__.

__barSizeSetting__: Time period of one bar. Must be one of:
        __'1 secs', '5 secs', '10 secs' 15 secs', '30 secs',
        '1 min', '2 mins', '3 mins', '5 mins', '10 mins', '15 mins',
        '20 mins', '30 mins',
        '1 hour', '2 hours', '3 hours', '4 hours', '8 hours',
        '1 day', '1 week', '1 month'__.

In [ ]:
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='100 S',
        barSizeSetting='10 secs',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)

In [ ]:
bars

In [ ]:
pd.DataFrame(bars)

In [ ]:
ib.cancelHistoricalData(bars) # cancel subscription

In [ ]:
def onBarUpdate(bars, hasNewBar):  # what shall happen after receiving a new bar
    global df
    df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]]
    df.set_index("date", inplace = True)
    clear_output(wait=True)
    display(df)

In [ ]:
# start stream
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='50 S',
        barSizeSetting='5 secs',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)

In [ ]:
bars.updateEvent += onBarUpdate # activate onBarUpdate
ib.sleep(30) # new to be added!!!

In [ ]:
df # still getting updated until you de-activate or cancel

In [ ]:
bars.updateEvent -= onBarUpdate # de-activate onBarUpdate

In [ ]:
ib.cancelHistoricalData(bars) # cancel subscription

## Application: Creating a live Candle Stick Chart 

In [ ]:
def onBarUpdate(bars, hasNewBar):
    plt.close()
    plot = util.barplot(bars, title = "EURUSD", upColor = "green", downColor = "red")
    clear_output(wait=True)
    display(plot)

In [ ]:
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1000 S',
        barSizeSetting='10 secs',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)

In [ ]:
bars.updateEvent += onBarUpdate # activate onBarUpdate
ib.sleep(30) # new to be added!!!

In [ ]:
bars.updateEvent -= onBarUpdate # de-activate onBarUpdate

In [ ]:
ib.cancelHistoricalData(bars) # cancel subscription

In [ ]:
ib.disconnect()

## Preparing the Data for Day Trading

- Problem: most recent bar is (typically) incomplete (until the end of the bar has been reached)
- In Trading: Take actions once a new bar is complete 

In [ ]:
from ib_async import * 
import pandas as pd
import numpy as np
import datetime as dt # old
from datetime import datetime, timezone # new
from IPython.display import display, clear_output
util.startLoop() 

In [ ]:
ib = IB()
ib.connect()

In [ ]:
contract = Forex('EURUSD')
contract

In [ ]:
datetime.now(timezone.utc) # new (Python 3.12)

In [ ]:
def onBarUpdate(bars, hasNewBar):
    print(datetime.now(timezone.utc)) # new (Python 3.12)

In [ ]:
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting='1 min',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)

In [ ]:
bars.updateEvent += onBarUpdate
ib.sleep(30) # new to be added!!!

In [ ]:
ib.cancelHistoricalData(bars) 

In [ ]:
def onBarUpdate(bars, hasNewBar): 
    global df
    df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] # remove current bar (incomplete)
    df.set_index("date", inplace = True)
    clear_output(wait=True)
    display(df)

In [ ]:
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting='1 min',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
bars.updateEvent += onBarUpdate 
ib.sleep(90) # new to be added!!!

In [ ]:
df # still getting updated until you cancel

In [ ]:
ib.cancelHistoricalData(bars) 

## Improving Code Efficiency

Idea: We need to run major parts of onBarUpdate only if a bar is complete (== new bar has been added)

In [ ]:
bars

In [ ]:
# check if current (incomplete) bar is more recent than the last complete bar
bars[-1].date > bars[-2].date

In [ ]:
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    if bars[-1].date > last_bar: # if bar completed / new bar
        last_bar = bars[-1].date
    
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        clear_output(wait=True)
        display(df)

In [ ]:
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting='1 min',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
last_bar = bars[-1].date # initialize last_bar
bars.updateEvent += onBarUpdate
ib.sleep(90) # new to be added!!!

In [ ]:
df # still getting updated until you cancel

In [ ]:
ib.cancelHistoricalData(bars) 

In [ ]:
ib.disconnect()

In [ ]:
df

In [ ]:
last_bar

## Define an SMA Day Trading Strategy

In [ ]:
from ib_async import * 
import pandas as pd
import numpy as np
#import datetime as dt
from IPython.display import display, clear_output
util.startLoop() 

In [ ]:
ib = IB()
ib.connect()

In [ ]:
contract = Forex('EURUSD')
contract

Strategy: SMA 50/200 (minutes) Crossover (needs to be backtested!!!)

In [ ]:
sma_s = 50
sma_l = 200

In [ ]:
def onBarUpdate(bars, hasNewBar): 
    global df, last_bar
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1]
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["sma_s"] = df.close.rolling(sma_s).mean()
        df["sma_l"] = df.close.rolling(sma_l).mean()
        df.dropna(inplace = True)
        df["position"] = np.where(df["sma_s"] > df["sma_l"], 1, -1 )
        ####################################################################
        
        # Action
        clear_output(wait=True)
        display(df)
        

In [ ]:
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting='1 min',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
last_bar = bars[-1].date
bars.updateEvent += onBarUpdate 
ib.sleep(120) # new to be added!!!

In [ ]:
df # still getting updated until you cancel

In [ ]:
ib.cancelHistoricalData(bars) 

In [ ]:
ib.disconnect()

## Trading

__Please run the following code only with your Paper Trading Account!!!__

__Check the Regular Trading Hours!!!__

In [5]:
from ib_async import * 
import pandas as pd
import numpy as np
#import datetime as dt
from IPython.display import display, clear_output
util.startLoop() 

In [6]:
ib = IB()
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [7]:
# strategy parameters
sma_s = 50
sma_l = 200
freq = "1 min"
units = 1000
contract = Forex('EURUSD') # for data streaming
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD") # for trading
ib.qualifyContracts(cfd)
conID = cfd.conId

In [8]:
conID

143916318

__barSizeSetting__: Time period of one bar. Must be one of:
        __'1 secs', '5 secs', '10 secs' 15 secs', '30 secs',
        '1 min', '2 mins', '3 mins', '5 mins', '10 mins', '15 mins',
        '20 mins', '30 mins',
        '1 hour', '2 hours', '3 hours', '4 hours', '8 hours',
        '1 day', '1 week', '1 month'__.

In [9]:
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["sma_s"] = df.close.rolling(sma_s).mean()
        df["sma_l"] = df.close.rolling(sma_l).mean()
        df.dropna(inplace = True)
        df["position"] = np.where(df["sma_s"] > df["sma_l"], 1, -1 )
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        pass

def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

In [10]:
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D', # must be sufficiently long!!! (200 * 1 min)
        barSizeSetting=freq,
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
last_bar = bars[-1].date
bars.updateEvent += onBarUpdate
ib.sleep(240) # new - to be added

KeyboardInterrupt: 

In [ ]:
### remember: Trading continues until you cancel!!!

In [12]:
ib.cancelHistoricalData(bars)  # cancel trading

In [13]:
df

,close,sma_s,sma_l,position
date,,,,
2024-12-02 01:34:00+00:00,1.053200,1.053307,1.054595,-1
2024-12-02 01:35:00+00:00,1.053410,1.053311,1.054577,-1
2024-12-02 01:36:00+00:00,1.053360,1.053314,1.054560,-1
2024-12-02 01:37:00+00:00,1.053320,1.053314,1.054542,-1
2024-12-02 01:38:00+00:00,1.053355,1.053314,1.054525,-1
...,...,...,...,...
2024-12-02 11:40:00+00:00,1.052745,1.052390,1.051401,1
2024-12-02 11:41:00+00:00,1.052725,1.052407,1.051411,1
2024-12-02 11:42:00+00:00,1.052710,1.052422,1.051421,1


In [14]:
ib.disconnect()

## Trade Reporting

In [15]:
from ib_async import * 
import pandas as pd
import numpy as np
#import datetime as dt
from datetime import datetime, timezone # new
from IPython.display import display, clear_output
util.startLoop()

In [16]:
ib = IB()
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [17]:
ib.fills()

[Fill(contract=CFD(conId=143916318, symbol='EUR', right='?', exchange='SMART', currency='USD', localSymbol='EUR.USD', tradingClass='EUR.USD'), execution=Execution(execId='0000e20f.674d4620.01.01', time=datetime.datetime(2024, 12, 2, 11, 45, 6, tzinfo=datetime.timezone.utc), acctNumber='DU9519531', exchange='SMART', side='BOT', shares=1000.0, price=1.05255, permId=291660412, clientId=1, orderId=177, liquidation=0, cumQty=1000.0, avgPrice=1.05255, orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2, pendingPriceRevision=False), commissionReport=CommissionReport(execId='0000e20f.674d4620.01.01', commission=2.0, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), time=datetime.datetime(2024, 12, 2, 11, 45, 6, tzinfo=datetime.timezone.utc))]

In [18]:
util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")

,time,side,cumQty,avgPrice
execId,,,,
0000e20f.674d4620.01.01,2024-12-02 11:45:06+00:00,BOT,1000.0,1.05255


In [19]:
util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")

,realizedPNL
execId,
0000e20f.674d4620.01.01,0.0


In [20]:
# strategy parameters
sma_s = 2
sma_l = 5
freq = "1 min"
units = 1000
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [21]:
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["sma_s"] = df.close.rolling(sma_s).mean()
        df["sma_l"] = df.close.rolling(sma_l).mean()
        df.dropna(inplace = True)
        df["position"] = np.where(df["sma_s"] > df["sma_l"], 1, -1 )
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)

In [22]:
# session_start = pd.to_datetime(dt.datetime.utcnow()).tz_localize("utc") # old
session_start = pd.to_datetime(datetime.now(timezone.utc))# new 
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting=freq,
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
last_bar = bars[-1].date
bars.updateEvent += onBarUpdate
ib.sleep(120) # new - to be added

,side,cumQty,avgPrice,realizedPNL,cumPNL
time,,,,,


KeyboardInterrupt: 

In [23]:
df # still getting updated until you cancel. Trading continues until you cancel!!!

,close,sma_s,sma_l,position
date,,,,
2024-12-02 01:34:00+00:00,1.053200,1.053307,1.054595,-1
2024-12-02 01:35:00+00:00,1.053410,1.053311,1.054577,-1
2024-12-02 01:36:00+00:00,1.053360,1.053314,1.054560,-1
2024-12-02 01:37:00+00:00,1.053320,1.053314,1.054542,-1
2024-12-02 01:38:00+00:00,1.053355,1.053314,1.054525,-1
...,...,...,...,...
2024-12-02 11:40:00+00:00,1.052745,1.052390,1.051401,1
2024-12-02 11:41:00+00:00,1.052725,1.052407,1.051411,1
2024-12-02 11:42:00+00:00,1.052710,1.052422,1.051421,1


In [24]:
report # still getting updated until you cancel. Trading continues until you cancel!!!

,side,cumQty,avgPrice,realizedPNL,cumPNL
time,,,,,


In [25]:
ib.cancelHistoricalData(bars) # cancel trading

In [26]:
ib.disconnect()

## Stop the Trading Session

Potential Triggers:
- stop at a certain time (e.g. 21:59 UTC time)
- stop once a certain profit/loss limit has been reached
- etc.

Required Actions:
- Stop Stream/Session
- Close Open Position (go neutral)

In [27]:
from ib_async import * 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone # new
from IPython.display import display, clear_output
util.startLoop()

In [28]:
ib = IB()
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [29]:
# strategy parameters
sma_s = 2
sma_l = 5
freq = "1 min"
units = 1000
end_time = dt.time(11, 5, 0) # stop condition
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [30]:
# dt.datetime.utcnow().time() #old

In [31]:
datetime.now(timezone.utc).time() # new

datetime.time(11, 46, 41, 459840)

In [32]:
datetime.now(timezone.utc).time() >= end_time # new

True

In [33]:
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["sma_s"] = df.close.rolling(sma_s).mean()
        df["sma_l"] = df.close.rolling(sma_l).mean()
        df.dropna(inplace = True)
        df["position"] = np.where(df["sma_s"] > df["sma_l"], 1, -1 )
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)
  

In [34]:
# start trading session
session_start = pd.to_datetime(datetime.now(timezone.utc))# new
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting=freq,
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
last_bar = bars[-1].date
bars.updateEvent += onBarUpdate
ib.sleep(30) # new - to be added (optional)

# stop trading session
while True:
    ib.sleep(5) # check every 5 seconds
    if datetime.now(timezone.utc).time() >= end_time: # if stop conditions has been met
        execute_trade(target = 0) # close open position 
        ib.cancelHistoricalData(bars) # stop stream
        ib.sleep(10)
        try:
            trade_reporting() # final reporting
        except:
            pass
        print("Session Stopped.")
        ib.disconnect()
        break
    else:
        pass

,side,cumQty,avgPrice,realizedPNL,cumPNL
time,,,,,


KeyboardInterrupt: 

## Trading other Strategies - Coding Challenge

In [35]:
from ib_async import * 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone # new
from IPython.display import display, clear_output
util.startLoop()

In [36]:
ib = IB()
ib.connect()

Peer closed connection. clientId 1 already in use?


KeyboardInterrupt: 

__Strategy 1__: Simple Contrarian Strategy (1min / window = 1)

## Stop here if you don´t want to see the solution!

###############################################################

In [37]:
# strategy parameters
freq = "1 min"
window = 1
units = 1000
end_time = dt.time(21, 59, 0) # stop condition
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

Task exception was never retrieved
future: <Task finished name='Task-32' coro=<IB.connectAsync() done, defined at /home/aldoran/.local/lib/python3.12/site-packages/ib_async/ib.py:2004> exception=TimeoutError()>
Traceback (most recent call last):
  File "/usr/lib/python3.12/asyncio/tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/futures.py", line 287, in __await__
    yield self  # This tells Task to wait for completion.
    ^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/tasks.py", line 385, in __wakeup
    future.result()
  File "/usr/lib/python3.12/asyncio/futures.py", line 198, in result
    raise exc
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/python3.12/asyncio/tasks.py", line 316, in __step_run_and_handle_result
    result = coro.throw(exc)
             ^^^^^^^^^^^^^^^
  File "/home/aldoran/.local/lib/python

ConnectionError: Not connected

In [38]:
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["returns"] = np.log(df["close"] / df["close"].shift())
        df["position"] = -np.sign(df.returns.rolling(window).mean())
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)
  

In [39]:
# start trading session
session_start = pd.to_datetime(datetime.now(timezone.utc))# new
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting=freq,
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
last_bar = bars[-1].date
bars.updateEvent += onBarUpdate
ib.sleep(30) # new - to be added (optional)

# stop trading session
while True:
    ib.sleep(5) # check every 5 seconds
    if datetime.now(timezone.utc).time() >= end_time: # if stop conditions has been met
        execute_trade(target = 0) # close open position 
        ib.cancelHistoricalData(bars) # stop stream
        ib.sleep(10)
        try:
            trade_reporting() # final reporting
        except:
            pass
        print("Session Stopped.")
        ib.disconnect()
        break
    else:
        pass

ConnectionError: Not connected

__Strategy 2__: Bollinger Bands SMA 20 (minutes) / 1 Standard Deviation

## Stop here if you don´t want to see the solution!

###############################################################

In [ ]:
from ib_async import * 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone # new
from IPython.display import display, clear_output
util.startLoop()

In [ ]:
ib = IB()
ib.connect()

In [ ]:
# strategy parameters
freq = "1 min"
sma = 20
dev = 1
units = 1000
end_time = dt.time(21, 59, 0) # stop condition
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [ ]:
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["SMA"] = df["close"].rolling(sma).mean()
        df["Lower"] = df["SMA"] - df["close"].rolling(sma).std() * dev
        df["Upper"] = df["SMA"] + df["close"].rolling(sma).std() * dev
        df["distance"] = df["close"] - df.SMA
        df["position"] = np.where(df["close"] < df.Lower, 1, np.nan)
        df["position"] = np.where(df["close"] > df.Upper, -1, df["position"])
        df["position"] = np.where(df.distance * df.distance.shift(1) < 0, 0, df["position"])
        df["position"] = df.position.ffill().fillna(0)
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)
  

In [ ]:
# start trading session
session_start = pd.to_datetime(datetime.now(timezone.utc))# new
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting=freq,
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
last_bar = bars[-1].date
bars.updateEvent += onBarUpdate
ib.sleep(30) # new - to be added (optional)

# stop trading session
while True:
    ib.sleep(5) # check every 5 seconds
    if datetime.now(timezone.utc).time() >= end_time: # if stop conditions has been met
        execute_trade(target = 0) # close open position 
        ib.cancelHistoricalData(bars) # stop stream
        ib.sleep(10)
        try:
            trade_reporting() # final reporting
        except:
            pass
        print("Session Stopped.")
        ib.disconnect()
        break
    else:
        pass

## Machine Learning Strategies (1) - Model Fitting

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier # added (from sklearn v. 1.7)

In [ ]:
data = pd.read_csv("five_minute.csv", parse_dates = ["time"], index_col = "time")
data

In [ ]:
data["returns"] = np.log(data.div(data.shift(1)))

In [ ]:
data.dropna(inplace = True)

In [ ]:
data["direction"] = np.sign(data.returns)
data

In [ ]:
lags = 2

In [ ]:
cols = []
for lag in range(1, lags + 1):
    col = "lag{}".format(lag)
    data[col] = data.returns.shift(lag)
    cols.append(col)
data.dropna(inplace = True)

++++++++++++++++++++++++++++

__Scaling/Standardizing Features (new)__

In [ ]:
means = data[cols].mean()
means

In [ ]:
stand_devs = data[cols].std()
stand_devs

In [ ]:
data[cols] = (data[cols]-means) / stand_devs
data

+++++++++++++++++++++++++++

In [ ]:
lm = OneVsRestClassifier(LogisticRegression(C = 1e6, max_iter = 100000)) # new (from sklearn v. 1.7)

In [ ]:
lm.fit(data[cols], data.direction)
lm

__In-Sample "Prediction"__

In [ ]:
data["pred"] = lm.predict(data[cols])
data

In [ ]:
hits = np.sign(data.direction * data.pred).value_counts()
hits

In [ ]:
hit_ratio = hits[1.0] / sum(hits)
hit_ratio

__Saving the model__

In [ ]:
lm

In [ ]:
import pickle

In [ ]:
pickle.dump(lm, open("logreg.pkl", "wb"))

__Saving the parameters (mean, std)__ NEW

In [ ]:
params = {"mu":means, "std":stand_devs}
params

In [ ]:
pickle.dump(params, open("params.pkl", "wb"))

## Machine Learning Strategies (2) - Implementation

In [ ]:
from ib_async import * 
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
import pickle
import datetime as dt
from datetime import datetime, timezone # new
from IPython.display import display, clear_output
util.startLoop()

In [ ]:
ib = IB()
ib.connect()

In [ ]:
# strategy parameters
freq = "5 mins"
lm = pickle.load(open("logreg.pkl", "rb"))
lags = 2
params = pickle.load(open("params.pkl", "rb"))
means = params["mu"]
stand_devs = params["std"]
units = 1000
end_time = dt.time(21, 59, 0) # stop condition
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [ ]:
lm

In [ ]:
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]]#.iloc[:-1] keep latest incomplete bar
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["returns"] = np.log(df["close"] / df["close"].shift())

        cols = []
        for lag in range(1, lags + 1):
            col = "lag{}".format(lag)
            df[col] = df.returns.shift(lag)
            cols.append(col)
        df.dropna(inplace = True)
        df[cols] = (df[cols] - means) / stand_devs
        
        df["position"] = lm.predict(df[cols])
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)

In [ ]:
# start trading session
session_start = pd.to_datetime(datetime.now(timezone.utc))# new
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting=freq,
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
last_bar = bars[-1].date
bars.updateEvent += onBarUpdate
ib.sleep(30) # new - to be added (optional)

# stop trading session
while True:
    ib.sleep(5) # check every 5 seconds
    if datetime.now(timezone.utc).time() >= end_time: # if stop conditions has been met
        execute_trade(target = 0) # close open position 
        ib.cancelHistoricalData(bars) # stop stream
        ib.sleep(10)
        try:
            trade_reporting() # final reporting
        except:
            pass
        print("Session Stopped.")
        ib.disconnect()
        break
    else:
        pass